# Water Pricing 

Water pricing is the most important and the simples instrument to encourage a more efficient use of irrigation in crop production.

 In a study publisched in 2015 by the University of Xin Jiang, Hohenheim and the Julius Kuhn-Institut analyze how it's possible to model a Bayesian Network to improve the water usage by variating the water price. 
 
 The study was done to improve the water usage in Northwest China, in the Tarim River basin who suffers of extreme water drought's periods.

My work is inspired by this study. I have modified a little bit the Bayesian Network and this is the result:


<img src="ProjectPiviFrancesco_Bayesian_network.png" alt="drawing" width="600"/>


The variables considered for this model are:
- **Water pricing**: water price based decided by the governament. It can has 3 values: low, medium and high. 
- **Agricultural extension services**: training farmes with regards to good crop management practices to improve yield levels, resource use efficiency and profit.
- **Subsidy**: subsidizing agricultural producers for implementing advanced irrigation technology.
- **Operation and maintenance**: Investments for running the water supply system including planning, construction, monitoring and repair of water storage distribution infrastructure.
- **Altering crop pattern**: Shifting towards crop with higher water productivity.
- **Aptimizing farm management practices**: Improving farm management practices to increase yield and minimize water losses.
- **Adopting advanced irrigation technology**: Adapting advanced irrigation technology such as sprinkler irrigation or drip irrigation.
- **Water use efficiency**:  The water use efficiency (WUE) is defined ad the economic yield of crops produced per unit of water. Three states correspond to low, medium, and high levels.

### Define the net

In [13]:
import numpy as np
import pandas as pd
from pgmpy.models import BayesianNetwork
from pgmpy.inference import VariableElimination
water_price_model=BayesianNetwork([
    ("WaterPrice","Operation and Maintenance"),
    ("WaterPrice","AlteringPattern"),
    ("Agricultural servicies","OptimizingManagement"),
    ("Agricultural servicies","AlteringPattern"),
    ("Agricultural servicies","NewTecnologies"),
    ("Subsidy","OptimizingManagement"),
    ("Subsidy","NewTecnologies"),
    ("Operation and Maintenance","NewTecnologies"),
    ("AlteringPattern","WaterEfficiency"),
    ("NewTecnologies","WaterEfficiency"),
    ("OptimizingManagement","WaterEfficiency"),
])



### Defining CPDs

In [14]:
from pgmpy.factors.discrete import TabularCPD

cpd_water_price=TabularCPD(
    variable="WaterPrice",variable_card=3,values=[[0.33],[0.34],[0.33]]
)
cpd_agricultural_servicies=TabularCPD(
    variable="Agricultural servicies",variable_card=2,values=[[0.93],
                                                              [0.07]]
)
cpd_subsidy=TabularCPD(
    variable="Subsidy", variable_card=2,values=[[0.20],
                                                [0.80]]
)
cpd_operation_and_maintenance=TabularCPD(
    variable="Operation and Maintenance", variable_card=3, values=[[0.7,0.2,0.3],
                                                                  [0.2,0.3,0.4],
                                                                  [0.1,0.5,0.3]],evidence=["WaterPrice"],evidence_card=[3]
)
cpd_altering_crop_pattern=TabularCPD(
    variable="AlteringPattern",variable_card=2,values=[[0.98,0.72,0.90,0.44,0.60,0.70],
                                                       [0.02,0.28,0.10,0.56,0.40,0.30]],evidence=["WaterPrice","Agricultural servicies"],evidence_card=[3,2]
)
cpd_optimizing_farm_management_practise=TabularCPD(
    variable="OptimizingManagement",variable_card=2,values=[[0.99,0.4,0.8,0.1],
                                                            [0.01,0.6,0.2,0.9]],evidence=["Agricultural servicies","Subsidy"],evidence_card=[2,2]
)
cpd_new_irrigation_tecnologies=TabularCPD(
    variable="NewTecnologies",variable_card=3,            values=[[0.99, 0.90,0.98,0.85,0.45,0.30,0.10,0.05,0.15,0.10,0.10,0.00],
                                                                  [0.01, 0.09,0.02,0.10,0.50,0.60,0.65,0.70,0.60,0.65,0.40,0.10],
                                                                  [0.00, 0.01,0.00,0.05,0.05,0.10,0.25,0.25,0.25,0.25,0.50,0.90]],
    evidence=["Operation and Maintenance","Agricultural servicies","Subsidy"],evidence_card=[3,2,2]
)
cpd_water_use_efficiency=TabularCPD(
    variable="WaterEfficiency",variable_card=3,     values=[[1.00,0.92,0.90,0.98,0.80,0.30,0.85,0.75,0.50,0.60,0.10,0.00],
                                                            [0.00,0.06,0.08,0.02,0.18,0.60,0.10,0.20,0.40,0.30,0.50,0.20],
                                                            [0.00,0.02,0.02,0.00,0.02,0.10,0.05,0.05,0.10,0.10,0.40,0.80]],
    evidence=["AlteringPattern","OptimizingManagement","NewTecnologies"],evidence_card=[2,2,3]
)


### Adding CPDs and cheking if the model is correct

In [15]:

water_price_model.add_cpds(
    cpd_water_price,cpd_agricultural_servicies,cpd_subsidy,cpd_operation_and_maintenance,cpd_altering_crop_pattern,cpd_optimizing_farm_management_practise,
    cpd_new_irrigation_tecnologies,cpd_water_use_efficiency
)
water_price_model.check_model()


True

In [16]:
for cpd in water_price_model.get_cpds():
    print("CPD of {variable}".format(variable=cpd.variable))
    print(cpd)

CPD of WaterPrice
+---------------+------+
| WaterPrice(0) | 0.33 |
+---------------+------+
| WaterPrice(1) | 0.34 |
+---------------+------+
| WaterPrice(2) | 0.33 |
+---------------+------+
CPD of Agricultural servicies
+---------------------------+------+
| Agricultural servicies(0) | 0.93 |
+---------------------------+------+
| Agricultural servicies(1) | 0.07 |
+---------------------------+------+
CPD of Subsidy
+------------+-----+
| Subsidy(0) | 0.2 |
+------------+-----+
| Subsidy(1) | 0.8 |
+------------+-----+
CPD of Operation and Maintenance
+------------------------------+---------------+---------------+---------------+
| WaterPrice                   | WaterPrice(0) | WaterPrice(1) | WaterPrice(2) |
+------------------------------+---------------+---------------+---------------+
| Operation and Maintenance(0) | 0.7           | 0.2           | 0.3           |
+------------------------------+---------------+---------------+---------------+
| Operation and Maintenance(1) | 0

In [17]:
water_price_model.local_independencies("AlteringPattern")

(AlteringPattern ⟂ Subsidy, NewTecnologies, Operation and Maintenance, OptimizingManagement | WaterPrice, Agricultural servicies)

In [18]:
water_inference = VariableElimination(water_price_model)

In [19]:
print('P(Operation and Maintenance|Waterprice=high)')
q_1 = water_inference.query(['Operation and Maintenance'],{"WaterPrice":2})
print(q_1)

P(Operation and Maintenance|Waterprice=high)


0it [00:00, ?it/s]

0it [00:00, ?it/s]

+------------------------------+----------------------------------+
| Operation and Maintenance    |   phi(Operation and Maintenance) |
+==============================+==================================+
| Operation and Maintenance(0) |                           0.3000 |
+------------------------------+----------------------------------+
| Operation and Maintenance(1) |                           0.4000 |
+------------------------------+----------------------------------+
| Operation and Maintenance(2) |                           0.3000 |
+------------------------------+----------------------------------+


In [20]:
print("P(Altering crop patterns| Water Pricing=high, Subsidy=high)")
q_2=water_inference.query(["NewTecnologies"],{"WaterPrice":2,"Subsidy":1})
print(q_2)

P(Altering crop patterns| Water Pricing=high, Subsidy=high)


  0%|          | 0/2 [00:00<?, ?it/s]

  0%|          | 0/2 [00:00<?, ?it/s]

+-------------------+-----------------------+
| NewTecnologies    |   phi(NewTecnologies) |
+===================+=======================+
| NewTecnologies(0) |                0.4099 |
+-------------------+-----------------------+
| NewTecnologies(1) |                0.4535 |
+-------------------+-----------------------+
| NewTecnologies(2) |                0.1367 |
+-------------------+-----------------------+


In [21]:
print("P(Water price|WaterUseEfficiency=high,Newtecnologies=medium)")
q_3=water_inference.query(["WaterPrice"],{"WaterEfficiency":2,"NewTecnologies":1})
print(q_3)


P(Water price|WaterUseEfficiency=high,Newtecnologies=medium)


  0%|          | 0/5 [00:00<?, ?it/s]

  0%|          | 0/5 [00:00<?, ?it/s]

+---------------+-------------------+
| WaterPrice    |   phi(WaterPrice) |
+===============+===================+
| WaterPrice(0) |            0.0901 |
+---------------+-------------------+
| WaterPrice(1) |            0.3131 |
+---------------+-------------------+
| WaterPrice(2) |            0.5968 |
+---------------+-------------------+


In [22]:
print("P(Subsidy|Water use efficiency=high")
q_4=water_inference.query(["Subsidy"],{"WaterEfficiency":2})
print(q_4)
#da rivedere non torna molto

P(Subsidy|Water use efficiency=high


  0%|          | 0/6 [00:00<?, ?it/s]

  0%|          | 0/6 [00:00<?, ?it/s]

+------------+----------------+
| Subsidy    |   phi(Subsidy) |
+============+================+
| Subsidy(0) |         0.0756 |
+------------+----------------+
| Subsidy(1) |         0.9244 |
+------------+----------------+


In [23]:
print("Agricultural extencion servicies| Water use efficiency=high")
q_5=water_inference.query(["Agricultural servicies "],{"WaterEfficiency":2})
print(q_5)
#da rivedere 

Agricultural extencion servicies| Water use efficiency=high


NetworkXError: The node Agricultural servicies  is not in the digraph.

In [ ]:
water_price_model.local_independencies("NewTecnologies")
water_price_model.local_independencies("AlteringPattern")


(AlteringPattern ⟂ OptimizingManagement, NewTecnologies, Operation and Maintenance, Subsidy | WaterPrice, Agricultural servicies)

In [ ]:
print("WaterPrice| Water use efficiency=high")
q_6=water_inference.query(variables=["WaterPrice"],evidence={"WaterEfficiency":2})
print(q_6)

WaterPrice| Water use efficiency=high


Eliminating: Subsidy: 100%|██████████| 6/6 [00:00<00:00, 346.49it/s]         

+---------------+-------------------+
| WaterPrice    |   phi(WaterPrice) |
+===============+===================+
| WaterPrice(0) |            0.0939 |
+---------------+-------------------+
| WaterPrice(1) |            0.3541 |
+---------------+-------------------+
| WaterPrice(2) |            0.5519 |
+---------------+-------------------+


In [ ]:

water_price_model.active_trail_nodes("WaterPrice","Agricultural servicies","AlteringPattern")

{'WaterPrice': {'AlteringPattern',
  'NewTecnologies',
  'Operation and Maintenance',
  'WaterEfficiency',
  'WaterPrice'}}